<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Persist, Deploy, and Score a PMML model to Predict Iris Species</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
  <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/customer-satisfaction-prediction/images/users_banner_2-03.png?raw=true" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook demonstrates how to store a sample Predictive Model Markup Language (PMML) model and score test data. 

Some familiarity with python is helpful. This notebook is compatible with Python 3.5.

You will use the **Iris** data set to predict the species of an iris flower. This data set contains measurements of the iris perianth flower. 

## Learning goals

You will learn how to:

-  Set up the Watson Machine Learning (WML) instance.
-  Deploy a PMML model.
-  Score the deployed model.


## Contents

1.	[Set up the environment](#setup)
2.	[Persist, deploy, and score a PMML model in the Watson Machine Learning (WML) repository](#scoring)
3.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">here</a>).
-  Install the <a href="https://pypi.python.org/pypi?%3Aaction=search&term=watson-machine-learning-client&submit=search" target="_blank" rel="noopener no referrer">watson-machine-learning-client</a> (**not required on Watson Studio**).


### 1.1 Set up the WML service instance

In this subsection, you will learn how to set up and obtain details of your WML service instance.

#### Install the `watson-machine-learning-client` package from pypi
**Note:** `watson-machine-learning-client` documentation can be found <a href="http://wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [1]:
!rm -rf $PIP_BUILD/watson-machine-learning-client

In [ ]:
#Install the WML client if required.
!pip install --upgrade watson-machine-learning-client 

Authenticate the WML service on the IBM Cloud.

**Tip**: Authentication information (your credentials) can be found in the <a href="https://console.bluemix.net/docs/services/service_credentials.html#service_credentials" target="_blank" rel="noopener no referrer">Service Credentials</a> tab of the service instance that you created on the IBM Cloud. <BR>If you cannot find the **instance_id** field in **Service Credentials**, click **New credential (+)** to generate new authentication information. 

**Action**: Enter your WML service instance credentials in the following cell.

In [3]:
# Enter your credentials here.
wml_credentials = {
    'url': 'https://ibm-watson-ml.mybluemix.net',
    'username': '***',
    'password': '***',
    'instance_id': '***'
}

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [6]:
wml_client = WatsonMachineLearningAPIClient(wml_credentials)

You cand check the details of your Watson Machine Learning service by calling the `service_instance.details` attribute.

In [7]:
# List information about your WML service instance.
print(wml_client.service_instance.details)

{'metadata': {'created_at': '2018-12-10T08:05:26.901Z', 'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/b4b6c696-172c-4164-8049-c0b621dbf3c9', 'modified_at': '2018-12-10T08:05:26.901Z', 'guid': 'b4b6c696-172c-4164-8049-c0b621dbf3c9'}, 'entity': {'deployments': {'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/b4b6c696-172c-4164-8049-c0b621dbf3c9/deployments'}, 'status': 'Active', 'account': {'name': "Jihyoung Kim's Account", 'type': 'PAYG', 'id': '4512e0f7a54c74e3c3742a68d5641ddb'}, 'owner': {'ibm_id': '310000SXE7', 'country_code': 'USA', 'email': 'jihyoungkim@us.ibm.com', 'beta_user': True, 'user_id': 'b811bd0c-9771-4b3f-b86e-2d76bdd4105f'}, 'source': 'Bluemix', 'published_models': {'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/b4b6c696-172c-4164-8049-c0b621dbf3c9/published_models'}, 'plan': 'standard', 'usage': {'gpu_count_k80': {'limit': 48, 'current': 0}, 'model_count': {'limit': 1000, 'current': 25}, 'gpu_count_v100': {'limit': 34, 'current': 0}, 

Obtain information about the stored models in your WML instance.

In [8]:
# List information about the stored models.
wml_client.repository.list_models()

------------------------------------  -------------------------------------------  ------------------------  -----------------
GUID                                  NAME                                         CREATED                   FRAMEWORK
c0e97a79-7db0-471e-afd8-f3e238fd16b6  XGBoost model for breast cancer              2019-02-28T19:11:24.484Z  scikit-learn-0.19
8008b47c-0f07-482c-8d2f-0b6940c824e5  SMS Spam prediction model                    2019-02-28T00:57:59.713Z  scikit-learn-0.19
f2ea0716-7d1f-484a-8468-14f4b6ec7333  WML Product Line Prediction Model            2019-02-27T13:50:04.628Z  mllib-2.3
fe1f7f54-b66f-46ed-8785-9fa5d129e02e  SPSS model for Churn prediction              2019-02-27T11:53:10.672Z  spss-modeler-18.1
7bd8d8b7-bb2b-4e71-95e6-502fec50657f  SPSS model for Churn prediction              2019-02-27T09:29:04.867Z  spss-modeler-18.1
c4fbb8ff-0e1b-4c45-b155-85b07527ecc3  SPSS model for Churn prediction              2019-02-26T13:13:42.220Z  spss-modeler-18.1


List deployed models in your WML service instance.

In [9]:
# List the deployed models.
wml_client.deployments.list()

------------------------------------  -------------------------------------------------------  ------  --------------  ------------------------  -----------------  -------------
GUID                                  NAME                                                     TYPE    STATE           CREATED                   FRAMEWORK          ARTIFACT TYPE
450f555b-1952-4ef1-90be-96a16f600345  Predict breast cancer                                    online  DEPLOY_SUCCESS  2019-02-28T19:11:28.862Z  scikit-learn-0.19  model
dd987565-3e04-4179-9cf9-546710fbadcd  Web scraping python function deployment                  online  DEPLOY_SUCCESS  2019-02-28T11:50:04.209Z  n/a                function
984ffa92-1c82-4b54-bb56-f6f85e915227  Web scraping python function deployment                  online  DEPLOY_SUCCESS  2019-02-27T17:50:16.907Z  n/a                function
e5472c3f-c7d7-4a3e-9b20-44f03030373e  Web scraping python function deployment                  online  DEPLOY_SUCCESS  2019-02-2

<a id="scoring"></a>
## 2. Persist, deploy, and score a PMML model in the Watson Machine Learning (WML) repository

In this section, you will use the Python API client for the WML service to create an online deployment and score the PMML model with a new data record.

For more information, please refer to the <a href="http://wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">Python API client documentation</a>.

- [2.1 Persist a PMML model](#persist)
- [2.2 Create an online deployment](#deploy)
- [2.3 Score a test data record](#score)


### 2.1 Persist a PMML model<a id="persist"></a>

**Action**: Use `wget` to download the sample PMML model, `iris_chaid.xml` from the <a href="https://github.com/pmservice/wml-sample-models" target="_blank" rel="noopener no referrer">Git repository</a>.  
**Note:** You may need to install the `wget` package. To install the `wget` package, run the following command. 

In [ ]:
!pip install --upgrade wget

In [11]:
# Download the sample PMML model and the 'iris_chaid.xml' file from the Github repo.
import wget
import os

sample_dir = 'pmml_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename=os.path.join(sample_dir, 'iris_chaid.xml')
if not os.path.isfile(filename):
    filename = wget.download('https://github.com/pmservice/wml-sample-models/raw/master/pmml/iris-species/model/iris_chaid.xml', out=sample_dir)

Store the downloaded file as *CHAID PMML model for Iris data* in the WML service repository, then list all the files stored in the repository.

In [12]:
# Save the downloaded file to the WML repository.
props_pmml = {
    wml_client.repository.ModelMetaNames.NAME: 'CHAID PMML model for Iris data',
    wml_client.repository.ModelMetaNames.AUTHOR_NAME: 'IBM', 
    wml_client.repository.ModelMetaNames.FRAMEWORK_NAME: 'pmml', 
    wml_client.repository.ModelMetaNames.FRAMEWORK_VERSION: '4.2'
}

model_details = wml_client.repository.store_model(filename, props_pmml)
wml_client.repository.list_models()

------------------------------------  -------------------------------------------  ------------------------  -----------------
GUID                                  NAME                                         CREATED                   FRAMEWORK
c9dfebfa-0411-4e3e-a9c5-9a311444ed59  CHAID PMML model for Iris data               2019-03-04T12:59:21.351Z  pmml-4.2
c0e97a79-7db0-471e-afd8-f3e238fd16b6  XGBoost model for breast cancer              2019-02-28T19:11:24.484Z  scikit-learn-0.19
8008b47c-0f07-482c-8d2f-0b6940c824e5  SMS Spam prediction model                    2019-02-28T00:57:59.713Z  scikit-learn-0.19
f2ea0716-7d1f-484a-8468-14f4b6ec7333  WML Product Line Prediction Model            2019-02-27T13:50:04.628Z  mllib-2.3
fe1f7f54-b66f-46ed-8785-9fa5d129e02e  SPSS model for Churn prediction              2019-02-27T11:53:10.672Z  spss-modeler-18.1
7bd8d8b7-bb2b-4e71-95e6-502fec50657f  SPSS model for Churn prediction              2019-02-27T09:29:04.867Z  spss-modeler-18.1
c4fbb8ff-

**Note:** From the list of downloaded files, you can see that model is successfully stored in the WML repository.

### 2.2 Create an online deployment<a id="deploy"></a>

Now, create an online deployment, *Iris species prediction*, for the stored model, and list all the online deployments for the model.

In [13]:
# Create an online deployment.
model_uid = wml_client.repository.get_model_uid(model_details)
model_deployment = wml_client.deployments.create(model_uid, name='Iris species prediction')

wml_client.deployments.list()



#######################################################################################

Synchronous deployment creation for uid: 'c9dfebfa-0411-4e3e-a9c5-9a311444ed59' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='ded4770f-c2fe-4653-9ce2-0c79ecfa10be'
------------------------------------------------------------------------------------------------


------------------------------------  -------------------------------------------------------  ------  --------------  ------------------------  -----------------  -------------
GUID                                  NAME                                                     TYPE    STATE           CREATED                   FRAMEWORK          ARTIFACT TYPE
ded4770f-c2fe-4653-9ce2-0c79ecfa10be  Iris species 

**Note:** From the list of deployed models, you can see that model was  successfully created.

### 2.3 Score a test data record<a id="score"></a>

Send a sample record using the Python API client to check if the online scoring endpoint is available.

In [ ]:
# Verify the online scoring endpoint.
model_scoring_url = wml_client.deployments.get_scoring_url(model_deployment)

print(model_scoring_url)

You can see that the endpoint url is available for sending scoring requests.

Now, you can score data and predict the species of iris flower.

In [15]:
# Score data and predict species of flower.
scoring_data = {
    'fields': ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width'], 
    'values': [[5.1, 3.5, 1.4, 0.2]]
}
predictions = wml_client.deployments.score(model_scoring_url, scoring_data) 

print(predictions)

{'fields': [['$R-Species', '$RC-Species', '$RP-Species', '$RP-setosa', '$RP-versicolor', '$RP-virginica', '$RI-Species']], 'values': [['setosa', 1.0, 1.0, 1.0, 0.0, 0.0, '1']]}


As we can see from the prediction, this is the Iris Setosa flower.

<a id="summary"></a>
## 3. Summary and next steps     

You successfully completed this notebook! You learned how to use Watson Machine Learning for PMML model deployment and scoring. 

Check out our <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">documentation</a> for more samples, tutorials, documentation, how-tos, and blog posts. 

### Citation

Dua, D. and Karra Taniskidou, E. (2017). [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml). Irvine, CA: University of California, School of Information and Computer Science.


### Authors

**Wojciech Sobala** is a Data Scientist at IBM.  
**Jihyoung Kim**, Ph.D., is a Data Scientist at IBM who strives to make data science easy for everyone through Watson Studio.

Copyright © 2018, 2019 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>